# 📘 Anleitung: Starten des Notebooks

Dieses Notebook wird von GitHub geladen. Um die Analyse zu starten, folgen Sie bitte diesen Schritten:

1. **Umgebung vorbereiten:** Gehen Sie oben im Menü auf **Laufzeit** (Runtime) → **Alle ausführen** (Run all).
2. **Warnung bestätigen:** Wenn das Fenster *"Warnung: Dieses Notebook wurde nicht von Google erstellt"* erscheint, klicken Sie auf **„Trotzdem ausführen“**.
3. **Berechnung starten:** Sobald das Bedienfeld erscheint, klicken Sie unten auf die grüne Schaltfläche **▶ Ausführen**.

---
*Die Warnung ist eine Google-Sicherheitsmaßnahme für GitHub-Dateien. Die grüne Schaltfläche trainiert das KI-Modell und sendet die Prognosen an die App.*

In [ ]:
# @title 📱 LOTTO CONTROL PANEL
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# ==========================================
# 1. UI DEFINITION (Das Interface)
# ==========================================

# Styling
style = """
<style>
.lotto-panel {
    background: #ffffff; border: 1px solid #e0e0e0; border-radius: 12px;
    padding: 15px; box-shadow: 0 4px 12px rgba(0,0,0,0.05); margin-bottom: 20px;
}
.lotto-header { font-size: 18px; font-weight: 700; color: #1a1a1a; margin-bottom: 12px; }
.lotto-controls { display: flex; gap: 10px; align-items: center; flex-wrap: wrap; }
.lotto-status { margin-top: 10px; font-size: 13px; color: #666; }
</style>
"""
display(HTML(style))

# Widgets
lbl_title = widgets.HTML('<div class="lotto-header">🎲 LOTTO 6aus49 Runner</div>')

txt_userid = widgets.Text(
    value="", placeholder="User ID (Auto)", description="🆔 ID:",
    layout=widgets.Layout(width="200px")
)

btn_run = widgets.Button(
    description="▶ Start Analysis",
    button_style="success", # gruen
    icon="play",
    layout=widgets.Layout(width="160px", height="36px")
)

btn_stop = widgets.Button(description="⏹", button_style="danger", layout=widgets.Layout(width="40px"))

progress = widgets.IntProgress(
    value=0, min=0, max=100,
    description="", bar_style="info",
    layout=widgets.Layout(width="100%", height="8px", margin="10px 0")
)

out_log = widgets.Output(layout={'border': '1px solid #eee', 'height': '200px', 'overflow_y': 'scroll', 'padding': '8px'})

# Container setup
ui_container = widgets.VBox([
    lbl_title,
    widgets.HBox([txt_userid, btn_run, btn_stop]),
    progress,
    out_log
])
ui_container.add_class("lotto-panel")

# --- ANZEIGEN (Ganz oben!) ---
display(ui_container)


# ==========================================
# 2. LOGIC & IMPORTS (Der "Motor")
# ==========================================
import time, requests, json, zipfile, io, os
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.callbacks import Callback

BACKEND_URL = "https://apilotto.euroceiling39.ru"

def log(msg, clear=False):
    if clear: out_log.clear_output()
    with out_log:
        print(f"[{time.strftime('%H:%M:%S')}] {msg}")

def get_hash_params():
    try:
        from google.colab import output
        js = """(function(){
          var h=window.location.hash.slice(1); var p={};
          h.split('&').forEach(function(v){ var i=v.split('='); p[i[0]]=decodeURIComponent(i[1]); });
          return p;
        })();"""
        return output.eval_js(js) or {}
    except: return {}

def pipeline(b):
    # 1. Setup
    btn_run.disabled = True
    progress.value = 0
    log("🚀 Starte Prozess...", clear=True)

    # Auto-ID Check if empty
    if not txt_userid.value:
        p = get_hash_params()
        if "userId" in p: txt_userid.value = str(p["userId"])

    uid = txt_userid.value.strip()
    if not uid:
        log("❌ FEHLER: Keine User ID! Bitte Link aus der App benutzen.");
        btn_run.disabled = False
        return

    try:
        # 2. Download
        progress.value = 10
        log(f"👤 User: {uid}")
        log("📥 Lade Lotto-Archiv...")

        r = requests.get("https://www.lotto-bayern.de/static/gamebroker_2/de/download_files/archiv_lotto.zip", timeout=30)
        with zipfile.ZipFile(io.BytesIO(r.content)) as z: z.extractall("/content/tmp")

        # 3. Parse
        log("📊 Verarbeite Daten...")
        rows = []
        with open("/content/tmp/lotto.txt", "r", encoding="utf-8", errors="ignore") as f:
            for l in f.readlines()[1:]:
                p = l.split()
                if len(p)>10 and int(p[2])>=2013: rows.append(p[3:10])

        df = pd.DataFrame(rows).apply(pd.to_numeric, errors='coerce').dropna()
        vals = df.values; vals[:,:6] = np.sort(vals[:,:6], axis=1)

        # 4. Train Model
        progress.value = 30
        log("🧠 Starte KI-Training (TensorFlow)...")

        scaler = StandardScaler().fit(vals)
        scaled = scaler.transform(vals)
        win=24
        X = np.array([scaled[i:i+win] for i in range(len(scaled)-win)])
        y = np.array([scaled[i+win] for i in range(len(scaled)-win)])

        model = Sequential([
            Bidirectional(LSTM(128, return_sequences=True), input_shape=(win,7)),
            Dropout(0.2), Bidirectional(LSTM(64)), Dense(7)
        ])
        model.compile(optimizer='adam', loss='mse')

        # Custom Callback for Progress Bar
        class Prog(Callback):
            def on_epoch_end(s,e,l): progress.value = 30 + int(60 * (e/60))

        model.fit(X, y, epochs=60, batch_size=64, verbose=0, callbacks=[Prog()])

        # 5. Predict
        last = scaled[-win:].reshape(1,win,7)
        pred = scaler.inverse_transform(model.predict(last))[0]
        nums = sorted([min(49,max(1,int(round(x)))) for x in pred[:6]])
        sz = min(9,max(0,int(round(pred[6]))))

        log(f"✅ Prognose: {nums} (SZ: {sz})")

        # 6. Send
        log(f"📡 Sende an App ({BACKEND_URL})...")
        res = requests.post(f"{BACKEND_URL}/api/colab/update", json={
            "userId": uid, "numbers": nums, "generated_at": int(time.time())
        })

        if res.ok:
            log("🎉 ERFOLG! Daten übertragen.")
            progress.value = 100
        else:
            log(f"⚠️ Server Fehler: {res.status_code}")

    except Exception as e:
        log(f"💥 Critical Error: {str(e)}")

    btn_run.disabled = False

# Linking
btn_run.on_click(pipeline)
btn_stop.on_click(lambda b: log("Stop requested (not impl)"))

# Auto-Run ID Check on Load
p = get_hash_params()
if "userId" in p: txt_userid.value = str(p["userId"])